In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# Matrix Factorization

Matrix factorization is a class of collaborative filtering algorithms used in recommendation systems. The goal of matrix factorization is to learn the latent preferences of users and the latent attributes of items from known ratings (learn the latent factors of users and items). Matrix factorization can be done by various techniques including Singular Value Decomposition (SVD) and Alternating Least Squares (ALS).

In [4]:
# Example user-item interactions (user_id, item_id, rating)
# In this example, we have 3 users and 5 items
# Entry 1: User 0 rates Item 0 with 5
# Entry 2: User 0 rates Item 1 with 3
# Entry 3: User 0 rates Item 2 with 1
interactions = [
    (0, 0, 5),
    (0, 1, 3),
    (0, 2, 1),
    (1, 0, 4),
    (1, 3, 2),
    (2, 1, 5),
    (2, 2, 4),
    (2, 4, 2),
]

In [9]:
# Convert to tensors
user_tensor = torch.LongTensor([x[0] for x in interactions])
item_tensor = torch.LongTensor([x[1] for x in interactions])
rating_tensor = torch.FloatTensor([x[2] for x in interactions])

In [10]:
print(user_tensor)
print(item_tensor)
print(rating_tensor)

tensor([0, 0, 0, 1, 1, 2, 2, 2])
tensor([0, 1, 2, 0, 3, 1, 2, 4])
tensor([5., 3., 1., 4., 2., 5., 4., 2.])


# Model Building of Matrix Factorization

Matrix factorization is a collaborative filtering technique that creates a recommendation model by learning compact representations (called embedding vectors) for each user and each item. These embedding vectors capture latent features, or hidden characteristics, that summarize patterns in user preferences and item attributes without needing explicit labels like "action" or "romance."

Each embedding vector represents a user or item in a lower-dimensional space, where each dimension relates to an underlying feature or preference. For instance, a user vector may capture a preference for "action movies" as a positive value on one dimension, while an item vector might have a similarly high score in that dimension if it’s an action movie.

To predict a rating, the model calculates the dot product of the user and item embedding vectors. This dot product measures the compatibility between a user and an item in the latent space. A high dot product indicates a strong match, while a low dot product suggests a weaker match.

During training, the model learns optimal embeddings by minimizing the mean squared error (MSE) between the predicted and actual ratings in the dataset. This optimization allows the model to adjust the embedding vectors to better represent user preferences and item characteristics, improving recommendation accuracy. Once trained, the model can predict ratings for new user-item pairs based on these learned embeddings, effectively generalizing user preferences to items they haven’t interacted with directly.

In [2]:
# Define the matrix factorization model
class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_items, num_factors):
        super(MatrixFactorization, self).__init__()
        # Embedding for users and items
        # Shape: (num_users, num_factors)
        self.user_factors = nn.Embedding(num_users, num_factors)  # User latent factors
        # Shape: (num_items, num_factors)
        self.item_factors = nn.Embedding(num_items, num_factors)  # Item latent factors

    def forward(self, user, item):
        # Dot product of user and item latent factors to get the predicted rating
        # Shape: (batch_size, num_factors)
        user_embedding = self.user_factors(user)
        # Shape: (batch_size, num_factors)
        item_embedding = self.item_factors(item)
        # Shape: (batch_size, num_factors) * (batch_size, num_factors) = (batch_size, 1)
        return (user_embedding * item_embedding).sum(1)

In [11]:
# Hyperparameters
num_users = 10  # Number of users
num_items = 15  # Number of items
num_factors = 5  # Number of latent factors
learning_rate = 0.01
num_epochs = 100

In [12]:
# Initialize the model, loss function, and optimizer
model = MatrixFactorization(num_users, num_items, num_factors)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass: compute predicted ratings
    predictions = model(user_tensor, item_tensor)

    # Compute the loss
    loss = criterion(predictions, rating_tensor)

    # Backward pass and optimization step
    loss.backward()
    optimizer.step()

    # Print loss for every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [10/100], Loss: 16.1182
Epoch [20/100], Loss: 12.4053
Epoch [30/100], Loss: 9.6322
Epoch [40/100], Loss: 7.3787
Epoch [50/100], Loss: 5.4465
Epoch [60/100], Loss: 3.8017
Epoch [70/100], Loss: 2.4587
Epoch [80/100], Loss: 1.4391
Epoch [90/100], Loss: 0.7383
Epoch [100/100], Loss: 0.3167


# Inferece

In [16]:
# Print the user and item latent factors
user_factors = model.user_factors.weight.data
item_factors = model.item_factors.weight.data

print("User factors:")
print(user_factors)

print("Item factors:")
print(item_factors)

User factors:
tensor([[-0.1966,  2.0901,  1.4320, -0.6804, -1.1663],
        [-0.7826,  1.8078, -0.4175, -0.4707, -0.8836],
        [-1.1315,  1.1421, -1.7936,  0.8216, -2.4299],
        [ 1.0879, -0.9965,  0.2485,  1.6590,  2.0396],
        [-0.4346,  0.1771, -0.9049,  0.2244,  0.8065],
        [-1.0381,  0.0959,  0.6811,  0.2722,  1.2688],
        [-0.6166, -0.4284, -0.8788, -0.4176,  0.7336],
        [-1.2199, -0.7998,  1.1784, -0.9541,  0.5998],
        [-0.0500,  0.1147,  0.0867,  0.4897,  0.4198],
        [-0.6322, -2.2283, -0.7515, -1.5604, -1.0636]])
Item factors:
tensor([[-0.9029,  1.2594,  1.1301,  0.1076, -0.4183],
        [ 0.8131,  0.5687, -0.6819, -0.4225, -1.6167],
        [-2.2512,  1.1665,  0.0447,  2.0653,  0.6119],
        [-0.0946,  1.1192, -0.7338,  0.5828,  0.0843],
        [-1.1505,  0.5486,  0.8121, -0.3932, -0.6636],
        [-0.7133,  0.6834, -0.8782,  0.5055,  0.8170],
        [ 2.4815, -1.3402, -0.3372, -0.8092, -0.2974],
        [ 0.8044, -0.7979, -0.2265, 

In [14]:
# Example of making predictions
model.eval()
with torch.no_grad():
    user_id = 0  # Example user
    item_id = 3  # Example item
    predicted_rating = model(torch.LongTensor([user_id]), torch.LongTensor([item_id]))
    print(
        f"Predicted rating for user {user_id} on item {item_id}: {predicted_rating.item():.4f}"
    )

Predicted rating for user 0 on item 3: 0.8122
